In [ ]:
import os

import pandas as pd
import numpy as np

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.theta import ThetaForecaster

import warnings
warnings.filterwarnings("ignore")

from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
warnings.simplefilter('ignore', ConvergenceWarning)
warnings.simplefilter('ignore', ValueWarning)


os.chdir(os.path.abspath(''))
df_train_total = pd.read_csv("../data/df_train_total.csv")
df_test_total = pd.read_csv("../data/df_test_total.csv")
df_merged = pd.read_csv("../data/df_merged.csv")

In [ ]:
def generate_ds_arima(label_df, input_seq_len=48, tau=12):
    col_labels =  'wl_1018680' # ['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']
    
    tmp_df = np.array(label_df.loc[label_df['month'].isin([2,3]), col_labels])
    
    n = tmp_df.shape[0] - input_seq_len - tau 
    
    conti_input = np.zeros((n, input_seq_len), dtype=np.float32)
    label = np.zeros((n, tau))

    for j in range(n):
        conti_input[j, :] = tmp_df[j:(j+input_seq_len)]
        label[j, :] = tmp_df[(j+input_seq_len):(j+input_seq_len+tau)]

    return conti_input, label

In [ ]:
arima_05_ql_results, arima_07_ql_results, arima_09_ql_results, arima_05_qr_results, arima_07_qr_results, arima_09_qr_results = [], [], [], [], [], []
theta_05_ql_results, theta_07_ql_results, theta_09_ql_results, theta_05_qr_results, theta_07_qr_results, theta_09_qr_results = [], [], [], [], [], []
lightgbm_05_ql_results, lightgbm_07_ql_results, lightgbm_09_ql_results, lightgbm_05_qr_results, lightgbm_07_qr_results, lightgbm_09_qr_results = [], [], [], [], [], []
ets_05_ql_results, ets_07_ql_results, ets_09_ql_results, ets_05_qr_results, ets_07_qr_results, ets_09_qr_results = [], [], [], [], [], []

### ARIMA

In [ ]:
for year in ["2016", "2017", "2018", "2019", "2020", "2021"]:
    df_train_total = pd.read_csv("../data/df_train_total_ds_{}.csv".format(year))
    df_test_total = pd.read_csv("../data/df_test_total_ds_{}.csv".format(year))
    df_merged = pd.read_csv("../data/df_merged_ds_{}.csv".format(year))

    fh = ForecastingHorizon(np.arange(1, 13), is_relative=True)

    arima_forecaster = ARIMA()

    arima_results = []

    y_train = df_merged.loc[df_merged["month"].isin([0, 1]), "wl_1018680"]
    arima_forecaster.fit(y_train)

    y_input, label = generate_ds_arima(df_merged)

    for i in range(y_input.shape[0]):
        arima_forecaster.update(y_input[i], update_params=False)
        tmp_result = arima_forecaster.predict_quantiles(fh=fh, alpha=[0.1, 0.5, 0.7, 0.9])
        
        arima_results.append(tmp_result.values[np.newaxis, ...])

    arima_results = np.concatenate(arima_results, axis=0)

    arima_09_ql_results.append(np.maximum(0.9 * (label - arima_results[..., 3]), (1-0.9)*(arima_results[..., 3] -label )).mean()/1000)
    arima_07_ql_results.append(np.maximum(0.7 * (label - arima_results[..., 2]), (1-0.7)*(arima_results[..., 2] -label )).mean()/1000)
    arima_05_ql_results.append(np.maximum(0.5 * (label - arima_results[..., 1]), (1-0.5)*(arima_results[..., 1] -label )).mean()/1000)
    
    arima_09_qr_results.append([np.mean(label < arima_results[..., 3]), np.abs(0.9 - np.mean(label < arima_results[..., 3]))])
    arima_07_qr_results.append([np.mean(label < arima_results[..., 2]), np.abs(0.7 - np.mean(label < arima_results[..., 2]))])
    arima_05_qr_results.append([np.mean(label < arima_results[..., 1]), np.abs(0.5 - np.mean(label < arima_results[..., 1]))])

In [ ]:
print(
    np.mean(arima_09_ql_results).round(4),
    np.mean(arima_07_ql_results).round(4),
    np.mean(arima_05_ql_results).round(4)
)

In [ ]:
print(
    np.mean(arima_09_qr_results, axis=0).round(4),
    np.mean(arima_07_qr_results, axis=0).round(4),
    np.mean(arima_05_qr_results, axis=0).round(4),
)

### LightGBM

In [ ]:
import lightgbm as lgb 
from sklearn.multioutput import MultiOutputRegressor

def generate_ts_data_train_for_lgb(label_df, term_list, input_seq_len=48, tau=12):
    conti_input_list = []
    label_list = []
    col_labels =  ['wl_1018680'] 

    for i in label_df['year'].unique():
        tmp_df = np.array(label_df.loc[label_df['month'].isin([0, 1]), :])
        tmp_label_df = np.array(label_df.loc[label_df['month'].isin([0, 1]), col_labels])        
        n = tmp_df.shape[0] - input_seq_len - tau 
        covariate = np.zeros((n, input_seq_len, tmp_df.shape[1] - 4 ))   
        label = np.zeros((n, tau, len(col_labels)))
        
        for j in range(n):
            covariate[j, :, :] = tmp_df[j:(j+input_seq_len), 4:]
            label[j, :, :] = tmp_label_df[(j+input_seq_len):(j+input_seq_len+tau), :]/1000

        conti_input_list.append(covariate)
        label_list.append(label)
    
    total_conti_input = np.concatenate(conti_input_list, axis=0)
    total_label = np.concatenate(label_list, axis=0)
    
    return total_conti_input.reshape(-1, total_conti_input.shape[1] * total_conti_input.shape[2]), np.squeeze(total_label)

def generate_ts_data_test_for_lgb(label_df, term_list, input_seq_len=48, tau=12):
    conti_input_list = []
    label_list = []
    col_labels =  ['wl_1018680'] 
    
    for i in label_df['year'].unique():
        tmp_df = np.array(label_df.loc[label_df['month'].isin([2, 3]), :])
        tmp_label_df = np.array(label_df.loc[label_df['month'].isin([2, 3]), col_labels])
        n = tmp_df.shape[0] - input_seq_len - tau 
    
        covariate = np.zeros((n, input_seq_len, tmp_df.shape[1] - 4 ))     
        label = np.zeros((n, tau, len(col_labels)))

        for j in range(n):
            covariate[j, :, :] = tmp_df[j:(j+input_seq_len), 4:]
            label[j, :, :] = tmp_label_df[(j+input_seq_len):(j+input_seq_len+tau), :]/1000

        conti_input_list.append(covariate)
        label_list.append(label)
    
    total_conti_input = np.concatenate(conti_input_list, axis=0)
    total_label = np.concatenate(label_list, axis=0)
    
    return total_conti_input.reshape(-1, total_conti_input.shape[1] * total_conti_input.shape[2]), np.squeeze(total_label)

In [ ]:
from sktime.transformations.series.fourier import FourierFeatures

for year in ["2016", "2017", "2018", "2019", "2020", "2021"]:

    df_train_total = pd.read_csv("../data/df_train_total_ds_{}.csv".format(year))
    df_test_total = pd.read_csv("../data/df_test_total_ds_{}.csv".format(year))
    df_merged = pd.read_csv("../data/df_merged_ds_{}.csv".format(year))

    term_list = 4
    transformer = FourierFeatures(sp_list=[24], fourier_terms_list=[term_list])

    df_merged_lgb = pd.concat([df_merged, transformer.fit_transform(df_merged[["wl_1018680"]])], axis=1)
    train_input_lgb, label_lgb = generate_ts_data_train_for_lgb(df_merged_lgb, term_list)

    alphas = [0.5, 0.7, 0.9]

    model_list = []

    for alpha in alphas:
        params = {
            "objective": "quantile",
            "alpha": alpha,
            "boosting": "gbdt",

        }
        gbm = lgb.LGBMRegressor(**params)
        regr_multiglb = MultiOutputRegressor(gbm)
        regr_multiglb.fit(train_input_lgb, label_lgb)
        
        model_list.append(
            regr_multiglb
        )

    test_input_lgb, test_label_lgb = generate_ts_data_test_for_lgb(df_merged_lgb, term_list)
    test_input_lgb.shape

    lgb_results_05 = model_list[0].predict(test_input_lgb)
    lgb_results_07 = model_list[1].predict(test_input_lgb)
    lgb_results_09 = model_list[2].predict(test_input_lgb)

    lightgbm_09_ql_results.append(np.maximum(0.9 * (test_label_lgb - lgb_results_09), (1-0.9)*(lgb_results_09 - test_label_lgb)).mean())
    lightgbm_07_ql_results.append(np.maximum(0.7 * (test_label_lgb - lgb_results_07), (1-0.7)*(lgb_results_07 - test_label_lgb)).mean())
    lightgbm_05_ql_results.append(np.maximum(0.5 * (test_label_lgb - lgb_results_05), (1-0.5)*(lgb_results_05 - test_label_lgb)).mean())

    lightgbm_09_qr_results.append([np.mean(test_label_lgb < lgb_results_09), np.abs(0.9 - np.mean(test_label_lgb < lgb_results_09))])
    lightgbm_07_qr_results.append([np.mean(test_label_lgb < lgb_results_07), np.abs(0.7 - np.mean(test_label_lgb < lgb_results_07))])
    lightgbm_05_qr_results.append([np.mean(test_label_lgb < lgb_results_05), np.abs(0.5 - np.mean(test_label_lgb < lgb_results_05))])

In [ ]:
print(
    np.mean(lightgbm_09_ql_results).round(4),
    np.mean(lightgbm_07_ql_results).round(4),
    np.mean(lightgbm_05_ql_results).round(4)
)


In [ ]:
print(
    np.mean(lightgbm_09_qr_results, axis=0).round(4),
    np.mean(lightgbm_07_qr_results, axis=0).round(4),
    np.mean(lightgbm_05_qr_results, axis=0).round(4),
)

### ETS

In [ ]:
for year in ["2016", "2017", "2018", "2019", "2020", "2021"]:
    df_train_total = pd.read_csv("../data/df_train_total_ds_{}.csv".format(year))
    df_test_total = pd.read_csv("../data/df_test_total_ds_{}.csv".format(year))
    df_merged = pd.read_csv("../data/df_merged_ds_{}.csv".format(year))

    fh = ForecastingHorizon(np.arange(1, 13), is_relative=True)

    y_train = df_merged.loc[df_merged["month"].isin([0, 1]), "wl_1018680"]
    y_input, label = generate_ds_arima(df_merged)

    ets_results = []
    ets_forecaster = AutoETS(auto=True, n_jobs=-1)  

    ets_forecaster.fit(y_train)

    for i in range(y_input.shape[0]):
        ets_forecaster.update(y_input[i], update_params=False)
        tmp_result = ets_forecaster.predict_quantiles(fh=fh, alpha=[0.1, 0.5, 0.7, 0.9])
        
        ets_results.append(tmp_result.values[np.newaxis, ...])
        
    ets_results = np.concatenate(ets_results, axis=0)

    ets_09_ql_results.append(np.maximum(0.9 * (label - ets_results[..., 3]), (1-0.9)*(ets_results[..., 3] -label )).mean()/1000)
    ets_07_ql_results.append(np.maximum(0.7 * (label - ets_results[..., 2]), (1-0.7)*(ets_results[..., 2] -label )).mean()/1000)
    ets_05_ql_results.append(np.maximum(0.5 * (label - ets_results[..., 1]), (1-0.5)*(ets_results[..., 1] -label )).mean()/1000)
    
    ets_09_qr_results.append([np.mean(label < ets_results[..., 3]), np.abs(0.9 - np.mean(label < ets_results[..., 3]))])
    ets_07_qr_results.append([np.mean(label < ets_results[..., 2]), np.abs(0.7 - np.mean(label < ets_results[..., 2]))])
    ets_05_qr_results.append([np.mean(label < ets_results[..., 1]), np.abs(0.5 - np.mean(label < ets_results[..., 1]))])

In [ ]:
print(
    np.mean(ets_09_ql_results).round(4),
    np.mean(ets_07_ql_results).round(4),
    np.mean(ets_05_ql_results).round(4)
)

In [ ]:
print(
    np.mean(ets_09_qr_results, axis=0).round(4),
    np.mean(ets_07_qr_results, axis=0).round(4),
    np.mean(ets_05_qr_results, axis=0).round(4),
)

### Theta

In [ ]:
for year in ["2016", "2017", "2018", "2019", "2020", "2021"]:
    df_train_total = pd.read_csv("../data/df_train_total_ds_{}.csv".format(year))
    df_test_total = pd.read_csv("../data/df_test_total_ds_{}.csv".format(year))
    df_merged = pd.read_csv("../data/df_merged_ds_{}.csv".format(year))

    fh = ForecastingHorizon(np.arange(1, 13), is_relative=True)

    y_train = df_merged.loc[df_merged["month"].isin([0, 1]), "wl_1018680"]
    y_input, label = generate_ds_arima(df_merged)

    theta_results = []
    theta_forecaster = ThetaForecaster()    

    theta_forecaster.fit(y_train)

    for i in range(y_input.shape[0]):
        theta_forecaster.update(y_input[i], update_params=False)
        tmp_result = theta_forecaster.predict_quantiles(fh=fh, alpha=[0.1, 0.5, 0.7, 0.9])
        
        theta_results.append(tmp_result.values[np.newaxis, ...])

    theta_results = np.concatenate(theta_results, axis=0)

    theta_09_ql_results.append(np.maximum(0.9 * (label - theta_results[..., 3]), (1-0.9)*(theta_results[..., 3] -label )).mean()/1000)
    theta_07_ql_results.append(np.maximum(0.7 * (label - theta_results[..., 2]), (1-0.7)*(theta_results[..., 2] -label )).mean()/1000)
    theta_05_ql_results.append(np.maximum(0.5 * (label - theta_results[..., 1]), (1-0.5)*(theta_results[..., 1] -label )).mean()/1000)
    
    theta_09_qr_results.append([np.mean(label < theta_results[..., 3]), np.abs(0.9 - np.mean(label < theta_results[..., 3]))])
    theta_07_qr_results.append([np.mean(label < theta_results[..., 2]), np.abs(0.7 - np.mean(label < theta_results[..., 2]))])
    theta_05_qr_results.append([np.mean(label < theta_results[..., 1]), np.abs(0.5 - np.mean(label < theta_results[..., 1]))])

In [ ]:
print(
    np.mean(theta_09_ql_results).round(4),
    np.mean(theta_07_ql_results).round(4),
    np.mean(theta_05_ql_results).round(4)
)

In [ ]:
print(
    np.mean(theta_09_qr_results, axis=0).round(4),
    np.mean(theta_07_qr_results, axis=0).round(4),
    np.mean(theta_05_qr_results, axis=0).round(4),
)